In [ ]:

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field, HttpUrl
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown
import requests
from typing import List, Optional
from datetime import date
from openai import OpenAI

In [ ]:
load_dotenv(override=True)

In [ ]:
instructions = f"You are a helpful web scraper. Given an html page, extract the data for the product. Be sure the data is in the JSON format. I will provide the structure of the JSON as a structured output."

class Product(BaseModel):
    product_name: str = Field(
        description="The full, official name of the product."
    )
    brand: str = Field(
        description="The manufacturer or brand of the product (e.g., 'Bandai Spirits')."
    )
    series: Optional[str] = Field(
        default=None,
        description="The specific series or franchise the product belongs to (e.g., 'Mobile Suit Gundam')."
    )
    category: Optional[str] = Field(
        default=None,
        description="The general category of the product (e.g., 'Model Kit', 'Action Figure')."
    )
    scale: Optional[str] = Field(
        default=None,
        description="The scale ratio of the model, if applicable (e.g., '1/100', '1/144')."
    )
    release_date: Optional[str] = Field(
        default=None,
        description="The original or scheduled release date of the product in YYYY-MM-DD format."
    )
    availability: str = Field(
        description="The current stock status (e.g., 'In Stock', 'Pre-order', 'Out of Stock')."
    )
    price: float = Field(
        description="The standard retail price of the product in the specified currency."
    )
    discounted_price: Optional[float] = Field(
        default=None,
        description="The sale or discounted price, if available."
    )
    currency: str = Field(
        description="The ISO 4217 currency code for the price (e.g., 'USD', 'JPY')."
    )
    shipping_cost: Optional[float] = Field(
        default=None,
        description="The cost of shipping, if a flat rate is listed. Exclude if it needs calculation."
    )
    store_location: Optional[str] = Field(
        default=None,
        description="The physical city and state/country of the store, if available."
    )
    stock_quantity: Optional[int] = Field(
        default=None,
        description="The number of items available in stock, if specified."
    )
    store_name: str = Field(
        description="The name of the online or physical store selling the product."
    )
    store_url: str = Field(
        description="The base URL or homepage of the store's website."
    )
    url: str = Field(
        description="The direct URL of the specific product page being viewed."
    )
    store_region: Optional[str] = Field(
        default=None,
        description="The geographic region the store primarily serves (e.g., 'USA', 'EU')."
    )
    images: List[str] = Field(
        description="A list of direct URLs to the product images."
    )
    description: str = Field(
        description="A detailed paragraph describing the product, including its features and any included accessories."
    )
    product_id: str = Field(
        description="The unique product identifier, SKU, model number, or barcode."
    )
# Agent which will output the search terms to use
# web_scraper_agent = Agent(
#     name="WebScraperAgent",
#     instructions=instructions,
#     model="gpt-4o-mini",
#     output_type=Product
# )

In [ ]:
# async def web_scraper(html_page : str):
#     """ Uses the web scraper agent to extract the data from the html page """
#     print("Planning searches...")
#     result = await Runner.run(web_scraper_agent, f"html_page: {html_page}")
#     print(result.final_output.model_dump_json())
#     return result.final_output

In [ ]:
html_page = """
<div data-product-featured="{&quot;id&quot;:&quot;8778809639086&quot;,&quot;isMainProduct&quot;:true, &quot;sectionId&quot;:&quot;template--18129367892142__main&quot;, &quot;disableSwatch&quot;:true, &quot;media&quot;: true,&quot;enableHistoryState&quot;: true, &quot;formID&quot;: &quot;#product-form-8778809639086template--18129367892142__main&quot;, &quot;removeSoldout&quot;:true, &quot;changeVariantByImg&quot;:true, &quot;isNoPick&quot;:false,&quot;hasSoldoutUnavailable&quot;:false,&quot;enable_zoom_click_mb&quot;:true,&quot;main_click&quot;:&quot;pswp&quot;,&quot;canMediaGroup&quot;:false,&quot;isGrouped&quot;:false,&quot;hasIsotope&quot;:false,&quot;available&quot;:true, &quot;customBadge&quot;:null, &quot;customBadgeHandle&quot;:null,&quot;dateStart&quot;:1750805934, &quot;compare_at_price&quot;:null,&quot;price&quot;:17995, &quot;isPreoder&quot;:false, &quot;showFirstMedia&quot;:false, &quot;isSticky&quot;:false, &quot;isStickyMB&quot;:false, &quot;stickyShow&quot;:&quot;2&quot;, &quot;useStickySelect&quot;: true }" class="t4s-row t4s-row__product is-zoom-type__inner2 initProducts__enabled" data-t4s-zoom-main="">

 <div class="t4s-col-md-6 t4s-col-12 t4s-col-item t4s-product__media-wrapper"><link href="//aztoyhobby.com/cdn/shop/t/28/assets/slider-settings.css?v=46721434436777892401742701639" rel="stylesheet" type="text/css" media="all">

 <div class="t4s-row t4s-g-0" timeline="" hdt-reveal="slide-in" observed="" animationend="">

 <div data-product-single-media-group="" class="t4s-col-lg t4s-order-lg-last t4s-col-12 t4s-col-item t4s-pr">

 <div data-t4s-gallery="" data-main-media="" data-t4s-thumb-true="" class="t4s-row t4s-g-0 t4s-slide-eff-fade flickityt4s t4s_ratio1_1 t4s_position_default t4s_contain t4s-flicky-slider t4s-slider-btn-true t4s-slider-btn-style-outline t4s-slider-btn-round t4s-slider-btn-small t4s-slider-btn-cl-dark t4s-slider-btn-vi-always t4s-slider-btn-hidden-mobile-true flickityt4s-enabled is-draggable is-fade t4s-enabled" data-flickityt4s-js="{&quot;t4sid&quot;: &quot;template--18129367892142__main&quot;, &quot;status&quot;: true, &quot;checkVisibility&quot;: false, &quot;cellSelector&quot;: &quot;[data-main-slide]:not(.is--media-hide)&quot;,&quot;isFilter&quot;:false,&quot;imagesLoaded&quot;: 0,&quot;adaptiveHeight&quot;: 1, &quot;contain&quot;: 1, &quot;groupCells&quot;: &quot;100%&quot;, &quot;dragThreshold&quot; : 6, &quot;cellAlign&quot;: &quot;left&quot;,&quot;wrapAround&quot;: true,&quot;prevNextButtons&quot;: true,&quot;percentPosition&quot;: 1,&quot;pageDots&quot;: false, &quot;autoPlay&quot; : 0, &quot;pauseAutoPlayOnHover&quot; : true , &quot;thumbNav&quot;: true, &quot;thumbVertical&quot;: true, &quot;isMedia&quot;: true }" style="--flickity-btn-pos: 0px;--flickity-btn-pos-mb: 0px;--space-dots: 10px;" tabindex="0"><div class="flickityt4s-viewport" style="height: 407.193px; touch-action: pan-y;"><div class="flickityt4s-slider" style="left: 0px; transform: translateX(0%);"><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item is-selected" data-media-id="31352879284398" data-nt-media-id="template--18129367892142__main-31352879284398" data-media-type="image" data-grname="" data-grpvl="" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.0460251046025104;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=576 576w" width="720" height="688" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174" class="t4s-lz--fadeIn lazyautosizes ls-is-cached lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201434%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1434" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879382702" data-nt-media-id="template--18129367892142__main-31352879382702" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.0416666666666667;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=576 576w" width="720" height="691" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176" class="t4s-lz--fadeIn lazyautosizes ls-is-cached lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201440%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1440" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879087790" data-nt-media-id="template--18129367892142__main-31352879087790" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.5511892450879008;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=576 576w" width="720" height="464" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177" class="t4s-lz--fadeIn lazyautosizes ls-is-cached lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%20967%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="967" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879415470" data-nt-media-id="template--18129367892142__main-31352879415470" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.374885426214482;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=576 576w" width="720" height="524" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180" class="t4s-lz--fadeIn lazyautosizes ls-is-cached lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201091%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1091" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879120558" data-nt-media-id="template--18129367892142__main-31352879120558" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.1773940345368916;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=576 576w" width="720" height="612" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201274%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1274" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879153326" data-nt-media-id="template--18129367892142__main-31352879153326" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.1971268954509178;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=576 576w" width="720" height="601" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201253%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1253" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879251630" data-nt-media-id="template--18129367892142__main-31352879251630" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.2396694214876034;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=576 576w" width="720" height="581" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201210%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1210" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879218862" data-nt-media-id="template--18129367892142__main-31352879218862" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.513622603430878;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=576 576w" width="720" height="476" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%20991%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="991" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879317166" data-nt-media-id="template--18129367892142__main-31352879317166" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:0.6806666666666666;--mw-media:1021px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=576 576w" width="720" height="1058" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201021%201500%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1021" height="1500" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1600 1600w" sizes="277.03133333333335px" srcset="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div><div data-product-single-media-wrapper="" data-main-slide="" class="t4s-col-12 t4s-col-item t4s-product__media-item " data-media-id="31352879349934" data-nt-media-id="template--18129367892142__main-31352879349934" data-media-type="image" data-grname="" data-grpvl="" aria-hidden="true" style="position: absolute; left: 0px; transform: translateX(0%);">

<div data-t4s-gallery--open="" class="t4s_ratio t4s-product__media" data-style="--aspect-ratioapt:1.0344827586206897;--mw-media:1500px">

<noscript><img src="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=720" alt="" srcset="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=288 288w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=576 576w" width="720" height="696" loading="lazy" class="t4s-img-noscript" sizes="(min-width: 1500px) 1500px, (min-width: 750px) calc((100vw - 11.5rem) / 2), calc(100vw - 4rem)"></noscript>

<img data-master="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190" class="t4s-lz--fadeIn lazyautosizes lazyloadt4sed" data-widths="[100,200,400,600,700,800,900,1000,1200,1400,1600]" data-optimumx="2" data-sizes="auto" src="data:image/svg+xml,%3Csvg%20viewBox%3D%220%200%201500%201450%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3C%2Fsvg%3E" width="1500" height="1450" alt="" data-srcset="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1600 1600w" sizes="407px" srcset="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=100 100w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=200 200w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=400 400w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=600 600w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=700 700w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=800 800w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=900 900w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1000 1000w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1200 1200w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1400 1400w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1600 1600w">

<span class="lazyloadt4s-loader"></span>

</div>

</div></div></div><button class="flickityt4s-button flickityt4s-prev-next-button previous" type="button" aria-label="Previous"><svg class="flickityt4s-button-icon" viewBox="0 0 100 100"><path d="M 10,50 L 60,100 L 70,90 L 30,50L 70,10 L 60,0 Z" class="arrow"></path></svg></button><button class="flickityt4s-button flickityt4s-prev-next-button next" type="button" aria-label="Next"><svg class="flickityt4s-button-icon" viewBox="0 0 100 100"><path d="M 10,50 L 60,100 L 70,90 L 30,50L 70,10 L 60,0 Z" class="arrow" transform="translate(100, 100) rotate(180) "></path></svg></button></div>

 <div data-product-single-badge="" data-sort="sale,new,soldout,preOrder,custom" class="t4s-single-product-badge t4s-pa t4s-pe-none t4s-op-0 lazyloadt4sed" data-rendert4s="css://aztoyhobby.com/cdn/shop/t/28/assets/single-pr-badge.css?v=8070179643820959801742701639" data-currentrendert4s=""><span data-badge-soldout="" class="t4s-badge-item t4s-badge-soldout" hidden="">Sold out</span><span data-badge-preorder="" class="t4s-badge-item t4s-badge-preorder" hidden="">Pre order</span></div></div><div class="t4s-col-lg-auto t4s-order-lg-first t4s-col-12 t4s-col-item t4s-col-thumb t4s-pr t4s-oh">

 <div data-thumb__scroller="template--18129367892142__main" class="t4s-carousel__nav-scroller is__position-left">

  <div class="t4s-row t4s-row-mt t4s-row-cols-4 t4s_ratio1_1 t4s_position_default t4s_contain t4s-g-5 t4s-carousel__nav carousel__nav--template--18129367892142__main carousel__nav-hover1 is--nav-ready"><div class="t4s-col-item t4s-carousel__nav-item is-nav-selected" data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes ls-is-cached lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item " data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=1752842188&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div><div class="t4s-col-item t4s-carousel__nav-item thumb-nav-visible-last" data-grname="" data-grpvl="" data-mdtype="image" data-vhost=""><div class="t4s_ratio t4s-carousel__nav-inner t4s-bg-11 t4s-child-lazyloaded" style="undefined;background: url(//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1)"><img alt="" loading="lazy" class="lazyautosizes lazyloadt4sed" data-widths="[80, 120, 160 ,180, 360, 540]" data-optimumx="1.8" data-sizes="auto" src="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=1" data-srcset="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=540 540w" sizes="80px" srcset="//aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=80 80w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=120 120w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=160 160w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=180 180w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=360 360w, //aztoyhobby.com/cdn/shop/files/81uwhqVWLfL._AC_SL1500.jpg?v=1752842190&amp;width=540 540w"><span class="t4s-thumbnail__badge t4s-not-style t4s-op-0" aria-hidden="true"></span></div></div></div>

 </div>

 <button type="button" data-thumb-btn__prev="template--18129367892142__main" aria-label="Previous" class="btn_pnav_prev t4s-pa t4s-pe-none t4s-op-0"></button>

 <button type="button" data-thumb-btn__next="template--18129367892142__main" aria-label="Next" class="btn_pnav_next t4s-pa t4s-pe-none t4s-op-0"></button>

 </div></div></div>

 <div data-t4s-zoom-info="" class="t4s-col-md-6 t4s-col-12 t4s-col-item t4s-product__info-wrapper t4s-pr">

 <div id="product-zoom-template--18129367892142__main" class="t4s-product__zoom-wrapper"></div>

 <div id="ProductInfo-template--template--18129367892142__main__main" class="t4s-product__info-container t4s-product__info-container--sticky" timeline="" hdt-reveal="slide-in" observed="" animationend="">

  <h1 class="t4s-product__title" style="--title-family:var(--font-family-1);--title-style:none;--title-size:16px;--title-weight:600;--title-line-height:1;--title-spacing:0px;--title-color:#222222;--title-color-hover:#222222;">Bandai Metal Robot Spirits Gundam Barbatos [1st - 4th Form] Action Figure</h1><div id="shopify-block-ANUNWYUY2bnVCckNTa__cd53cf50-5eb7-4a18-b7c2-59781f0924bd" class="shopify-block shopify-app-block gw-full-width">

<div class="gw-rv-average-placeholder" data-product_id="8778809639086" data-gw-product-id="8778809639086" style="display: block"></div>









</div><div class="t4s-product__price-review" style="--price-size:22px;--price-weight:400;--price-color:#696969;--price-sale-color:#ec0101;">

<div class="t4s-product-price" data-pr-price="" data-product-price="" data-saletype="2">$179.95

</div></div><div class="t4s-product__policies t4s-rte" data-product-policies=""><a href="/policies/shipping-policy">Shipping</a> calculated at checkout.

</div>

  <div id="t4s-stock35033d98-aec0-4370-999c-b6725bc14ab7" class="t4s-inventory_qty t4s-text-start" data-inventory-qty="{ &quot;reduce&quot;:false, &quot;inventoryQty&quot;:1, &quot;id&quot;:46622506025134, &quot;stock&quot;:&quot;1&quot;, &quot;qty&quot;:2, &quot;total&quot;:100, &quot;min&quot;:12, &quot;max&quot;:20, &quot;bgprocess&quot;:&quot;#f76b6a&quot;, &quot;bgten&quot;:&quot;#ec0101&quot; }" data-prid="8778809639086" data-ttcalc="100">

  <p data-message="" class="t4s-inventory_message t4s-dn t4s-ch t4s-lh-1 t4s-fwm" style="font-size: 15px; display: block;">THE LAST<span data-count="" class="t4s-count" style="background-color: rgb(255, 255, 255); color: rgb(247, 107, 106);">1</span> !</p></div><div id="shopify-block-ATStsQ01EekFmNWx3a__growave_loyalty_wishlist_block_product_earn_points_widget_HdAh3r" class="shopify-block shopify-app-block gw-full-width">

<div class="gw-rd-earn-points-product-page-widget-placeholder" data-gw-product-id="8778809639086" data-gw-variant-id="46622506025134" style="--gw-background-color:#ffffff;

 --gw-font-color:#000000;

 --gw-icon-color: #ff0000;



 --gw-icon-display: flex;

 

 display:block"></div>









</div><div class="t4s-product__description t4s-rte"><p>The Metal Robot Spirits Gundam Barbatos brings this iconic mecha to life with supervision from designer Naohiro Washio. Featuring a sturdy diecast frame and finely detailed paintwork, it offers a realistic...<button class="t4s-product__description_readm" data-go-id="#t4s-tab-destemplate--18129367892142__main">Read more</button></p></div><div class="t4s-product-form__variants is-no-pick__false is-payment-btn-true t4s-payment-button t4s-btn-color-darkis-remove-soldout-true is-btn-full-width__false is-btn-atc-txt-3 is-btn-ck-txt-3 is--fist-ratio-false" style=" --wishlist-color: #222222;--wishlist-hover-color: #56cfe1;--wishlist-active-color: #e81e1e;--compare-color: #222222;--compare-hover-color: #56cfe1;--compare-active-color: #222222;">

<div data-callbackvariant="" id="t4s-callBackVariantproduct-form-8778809639086template--18129367892142__main"><form method="post" action="/cart/add" id="product-form-8778809639086template--18129367892142__main" accept-charset="UTF-8" class="t4s-form__product is--main-sticky" enctype="multipart/form-data" data-productid="8778809639086" novalidate="novalidate" data-type="add-to-cart-form" data-disable-swatch="true" data-np-autofill-form-type="other" data-np-checked="1" data-np-watching="1"><input type="hidden" name="form_type" value="product"><input type="hidden" name="utf8" value="✓"><shopify-payment-terms variant-id="46622506025134" shopify-meta="{&quot;type&quot;:&quot;product&quot;,&quot;currency_code&quot;:&quot;USD&quot;,&quot;country_code&quot;:&quot;US&quot;,&quot;variants&quot;:[{&quot;id&quot;:46622506025134,&quot;price_per_term&quot;:&quot;$44.98&quot;,&quot;full_price&quot;:&quot;$179.95&quot;,&quot;eligible&quot;:true,&quot;available&quot;:true,&quot;number_of_payment_terms&quot;:4}],&quot;min_price&quot;:&quot;$35.00&quot;,&quot;max_price&quot;:&quot;$30,000.00&quot;,&quot;financing_plans&quot;:[{&quot;min_price&quot;:&quot;$35.00&quot;,&quot;max_price&quot;:&quot;$49.99&quot;,&quot;terms&quot;:[{&quot;apr&quot;:0,&quot;loan_type&quot;:&quot;split_pay&quot;,&quot;installments_count&quot;:2}]},{&quot;min_price&quot;:&quot;$50.00&quot;,&quot;max_price&quot;:&quot;$149.99&quot;,&quot;terms&quot;:[{&quot;apr&quot;:0,&quot;loan_type&quot;:&quot;split_pay&quot;,&quot;installments_count&quot;:4}]},{&quot;min_price&quot;:&quot;$150.00&quot;,&quot;max_price&quot;:&quot;$999.99&quot;,&quot;terms&quot;:[{&quot;apr&quot;:0,&quot;loan_type&quot;:&quot;split_pay&quot;,&quot;installments_count&quot;:4},{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:3},{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:6},{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:12}]},{&quot;min_price&quot;:&quot;$1,000.00&quot;,&quot;max_price&quot;:&quot;$30,000.00&quot;,&quot;terms&quot;:[{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:3},{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:6},{&quot;apr&quot;:15,&quot;loan_type&quot;:&quot;interest&quot;,&quot;installments_count&quot;:12}]}],&quot;installments_buyer_prequalification_enabled&quot;:true,&quot;seller_id&quot;:28251}" data-instance-id="1265ca85-ce60-41ad-b47b-87c5d2f418cc"></shopify-payment-terms><input name="id" value="46622506025134" type="hidden"><link href="//aztoyhobby.com/cdn/shop/t/28/assets/button-style.css?v=14170491694111950561742701639" rel="stylesheet" type="text/css" media="all">

<link href="//aztoyhobby.com/cdn/shop/t/28/assets/custom-effect.css?v=68323854132956845141742701639" rel="stylesheet" media="all" onload="this.media='all'"><!--Show instore only product-->



<div class="t4s-product-form__buttons" style="--pr-btn-round:40px;">

 <div class="t4s-d-flex t4s-flex-wrap"><div data-quantity-wrapper="" class="t4s-quantity-wrapper t4s-product-form__qty">

  <button data-quantity-selector="" data-decrease-qty="" type="button" class="t4s-quantity-selector is--minus"><svg focusable="false" class="icon icon--minus" viewBox="0 0 10 2" role="presentation"><path d="M10 0v2H0V0z" fill="currentColor"></path></svg></button>

  <input data-quantity-value="" type="number" class="t4s-quantity-input" step="1" min="1" max="1" name="quantity" value="1" size="4" pattern="[0-9]*" inputmode="numeric" data-np-intersection-state="visible">

  <button data-quantity-selector="" data-increase-qty="" type="button" class="t4s-quantity-selector is--plus"><svg focusable="false" class="icon icon--plus" viewBox="0 0 10 10" role="presentation"><path d="M6 4h4v2H6v4H4V6H0V4h4V0h2v4z" fill="currentColor" fill-rule="evenodd"></path></svg></button>

 </div><!-- render t4s_wis_cp.liquid --><button data-animation-atc="{ &quot;ani&quot;:&quot;none&quot;,&quot;time&quot;:6000 }" type="submit" name="add" data-atc-form="" class="t4s-product-form__submit t4s-btn t4s-btn-base t4s-btn-style-default t4s-btn-color-primary t4s-w-100 t4s-justify-content-centert4s-btn-effect-sweep-to-top t4s-btn-loading__svg" style="pointer-events: all; opacity: 1;"><span class="t4s-btn-atc_text">Add to cart</span>

 <span class="t4s-loading__spinner" hidden="">

  <svg width="16" height="16" hidden="" class="t4s-svg-spinner" focusable="false" role="presentation" viewBox="0 0 66 66" xmlns="http://www.w3.org/2000/svg"><circle class="t4s-path" fill="none" stroke-width="6" cx="33" cy="33" r="30"></circle></svg>

 </span>

 </button></div>



 <div style="display: block; margin-top: 15px;" class="gw-wl-add-to-wishlist-placeholder" data-gw-product_id="8778809639086"></div>



 <div class="gw-gr-add-to-registry-product-page-widget-placeholder" data-gw-product-id="8778809639086" style="display: block; margin-top: 15px;"><div class="gw-gr-add-to-registry-product-page-widget svelte-e13h2p"><button class="gw-btn gw-button --outlined --medium gw-btn--block" type="button" aria-disabled="false" variant="outlined"><div class="gw-button__loading"><div class="gw-spinner" style="--gw-spinner-size: 18px; --gw-spinner-color: #ccc"><svg class="gw-spinner__inner svelte-esm0lf" viewBox="0 0 50 50"><circle class="path svelte-esm0lf" cx="25" cy="25" r="20" fill="none" stroke-width="5"></circle></svg></div></div>Add to registry</button></div></div><div data-shopify="payment-button" class="shopify-payment-button"> <shopify-accelerated-checkout recommended="{&quot;name&quot;:&quot;shop_pay&quot;,&quot;wallet_params&quot;:{&quot;shopId&quot;:55218045102,&quot;merchantName&quot;:&quot;A-Z Toy Hobby&quot;,&quot;personalized&quot;:true}}" fallback="{&quot;name&quot;:&quot;buy_it_now&quot;,&quot;wallet_params&quot;:{}}" access-token="25059c68d6f654d153dd338510901212" buyer-country="US" buyer-locale="en" buyer-currency="USD" variant-params="[{&quot;id&quot;:46622506025134,&quot;requiresShipping&quot;:true}]" shop-id="55218045102" enabled-flags="[&quot;1b55bc27&quot;]" requires-shipping=""><shop-pay-wallet-button access-token="25059c68d6f654d153dd338510901212" buyer-country="US" buyer-currency="USD" wallet-params="{&quot;shopId&quot;:55218045102,&quot;merchantName&quot;:&quot;A-Z Toy Hobby&quot;,&quot;personalized&quot;:true}" page-type="product" slot="button" requires-shipping="" call-to-action="" is-shop-pay-personalization-eligible="true" style="pointer-events: all; opacity: 1;">&nbsp;</shop-pay-wallet-button><more-payment-options-link buyer-country="US" access-token="25059c68d6f654d153dd338510901212" recommended-instrument="ShopPay" slot="more-options" style="pointer-events: all; opacity: 1;"><a class="shopify-payment-button__more-options" id="more-payment-options-link" href="#">More payment options</a><style>#more-payment-options-link{cursor:pointer}</style></more-payment-options-link></shopify-accelerated-checkout> <small id="shopify-buyer-consent" class="hidden" aria-hidden="true"> This item is a recurring or deferred purchase. By continuing, I agree to the <span id="shopify-subscription-policy-button">cancellation policy</span> and authorize you to charge my payment method at the prices, frequency and dates listed on this page until my order is fulfilled or I cancel, if permitted. </small> </div></div>

<!--Show instore only product endif--><input type="hidden" name="product-id" value="8778809639086"><input type="hidden" name="section-id" value="template--18129367892142__main"></form></div>

</div><div id="shopify-block-Ab0RveEFmK01RSGxDU__avada_order_limits_app_block_EVY9jK" class="shopify-block shopify-app-block"><div class="Avada-OrderLimit-Block__Container"><div class="Avada-OrderLimit__ErrorMessage-Product Avada-OrderLimit__ErrorMessage-Product-46622506025134" style="display: none;"></div></div>

</div><div class="t4s-custom_7448ba19-9fc8-43d5-b355-6f4ac30cbf2d t4s-pr__html t4s-rte"><p><strong>Note:&nbsp;</strong>The products shown in photos have been professionally painted. Color swatches are for informational purposes only and may differ from the actual merchandise. Colors on your screen are also affected by monitor settings and browser.</p></div><div class="t4s-product_social-share t4s-text-end"><link href="//aztoyhobby.com/cdn/shop/t/28/assets/icon-social.css?v=92386074729244038331742701639" rel="stylesheet" type="text/css" media="all">

</div></div>

 </div>

</div>
"""

# with trace("WebScraperAgent"):
#     print("Starting Scraping...")
#     search_plan = await web_scraper(html_page)

In [47]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [51]:
completion = client.chat.completions.create(
  model="openai/gpt-4o-mini",
  messages=[
    {
        "role": "system", 
        "content": "You are a helpful web scraper. Given an html page, extract the data for the product. Be sure the data is in the JSON format. Use the extract_product_info tool to extract the data."},
    {
      "role": "user",
      "content": html_page,
    },
  ],
  tools=[
      {
          "type": "function",
          "function": {
              "name": "extract_product_info",
              "description": "Extracts key details about a product.",
              "parameters": Product.model_json_schema()
          }
      }
  ],
  tool_choice={"type": "function", "function": {"name": "extract_product_info"}}
)

tool_call = completion.choices[0].message.tool_calls[0]
json_output_string = tool_call.function.arguments

print(json_output_string)


{"product_name":"Bandai Metal Robot Spirits Gundam Barbatos [1st - 4th Form] Action Figure","brand":"Bandai","availability":"In Stock","price":179.95,"currency":"USD","store_name":"A-Z Toy Hobby","store_url":"//aztoyhobby.com","url":"//aztoyhobby.com/products/bandai-metal-robot-spirits-gundam-barbatos-1st-4th-form-action-figure","images":["//aztoyhobby.com/cdn/shop/files/81FLQ1catbL._AC_SL1500.jpg?v=1752842174","//aztoyhobby.com/cdn/shop/files/81A_xKUyocL._AC_SL1500.jpg?v=1752842176","//aztoyhobby.com/cdn/shop/files/71m2minUp3L._AC_SL1500.jpg?v=1752842177","//aztoyhobby.com/cdn/shop/files/71ZdssyZ06L._AC_SL1500.jpg?v=1752842180","//aztoyhobby.com/cdn/shop/files/81VqYGGzDuL._AC_SL1500.jpg?v=1752842181","//aztoyhobby.com/cdn/shop/files/71fZ6HJ2VIL._AC_SL1500.jpg?v=1752842183","//aztoyhobby.com/cdn/shop/files/714m_ebYtTL._AC_SL1500.jpg?v=1752842185","//aztoyhobby.com/cdn/shop/files/71cVWEgp1qL._AC_SL1500.jpg?v=1752842186","//aztoyhobby.com/cdn/shop/files/81KxAUgkXJL._AC_SL1500.jpg?v=17528